In [1]:
import numpy as np
import scipy.linalg
np.seterr(divide='ignore')
import itertools

In [2]:
def read_file(filename):
    with open(filename) as f:
        lines = [line.strip().replace("@", ",") for line in f]
    return np.loadtxt(lines, dtype=float, delimiter=',')

In [72]:
# M = read_file("../input/day24-sample1.txt")
M = read_file("../input/day24-input.txt")

In [73]:
N = M.shape[0]
# M

In [74]:
def intersection(x, u, y, v):
    a = np.array([u, -v]).T
    if np.linalg.det(a) == 0:
        return None
    b = y - x
    t = np.linalg.solve(a, b)
    
    if np.any(t < 0):
        return None
    return x + t[0] * u

In [75]:
def count_intersections(M, xmin, xmax):
    cnt = 0
    for i in range(N):
        y = M[i][:2]
        v = M[i][3:5]
        for j in range(i + 1, N):
            if j == i:
                continue
            x = M[j,:2]
            u = M[j,3:5]
            s = intersection(x, u, y, v)
            if s is not None and np.all(s >= xmin) and np.all(s <= xmax):
                cnt += 1
    return cnt

In [76]:
# minval = 7
# maxval = 27
minval = 200000000000000
maxval = 400000000000000

xmin = np.ones(2) * minval
xmax = np.ones(2) * maxval

part1 = count_intersections(M, xmin, xmax)
part1

14672

In [77]:
# I need 3 hailstones with linear independent velocities
# then, given that xi is the ith hailstone's origin and vi its velocity, 
# and given that the rock's origin and velocity are y and w,
# y × (vi - vj) - w × (xi - xj) = xi × vi + xj × vj
# holds.
# (6 unknowns means we can solve this using 2 hailstone pairs)
#
# y[1] * dv[2] - y[2] * dv[1] - w[1] * dx[2] + w[2] * dx[1] 
#      = xi[1] * vi[2] - xi[2] * vi[1] + xj[1] * vj[2] - xj[2] * vj[1]
#
# y[2] * dv[0] - y[0] * dv[2] - w[2] * dx[0] + w[0] * dx[2]
#      = xi[2] * vi[0] - xi[0] * vi[2] + xj[2] * vj[0] - xj[0] * vj[2]
#
# y[0] * dv[1] - y[1] * dv[0] - w[0] * dx[1] + w[1] * dx[0]
#      = xi[0] * vi[1] - xi[1] * vi[0] + xj[0] * vj[1] - xj[1] * vj[0]
#
# variables:
# y[0] y[1] y[2] v[0] v[1] v[2]
#



# let's hope the first 3 velocities are linear independent
# edit: they are not, we're going with 0, 1, 3
if np.linalg.det(M[:,3:6][[0, 1, 3]]) == 0:
    print("vectors are not independent")
    
a = np.zeros((6, 6))
b = np.zeros(6)

for idx, (hail1, hail2) in enumerate([[0, 1], [0, 2]]):
    xi = M[hail1][0:3]
    vi = M[hail1][3:6]
    xj = M[hail2][0:3]
    vj = M[hail2][3:6]
    
    dx = xi - xj
    dv = vi - vj

    for i in range(3):
        j = (i + 1) % 3
        k = (i + 2) % 3
        a[3 * idx + i][j] = -dv[k]
        a[3 * idx + i][k] = dv[j]
        a[3 * idx + i][3 + j] = dx[k]
        a[3 * idx + i][3 + k] = -dx[j]
        b[3 * idx + i] = -(xi[j] * vi[k] - xi[k] * vi[j] - xj[j] * vj[k] + xj[k] * vj[j])
print(a)
print(b)
np.linalg.solve(a, b)

[[ 0.00000000e+00 -3.80000000e+01 -6.00000000e+00  0.00000000e+00
  -6.71179061e+13  1.01235839e+14]
 [ 3.80000000e+01  0.00000000e+00 -2.19000000e+02  6.71179061e+13
   0.00000000e+00 -1.46633652e+14]
 [ 6.00000000e+00  2.19000000e+02  0.00000000e+00 -1.01235839e+14
   1.46633652e+14  0.00000000e+00]
 [ 0.00000000e+00  3.40000000e+01 -9.60000000e+01  0.00000000e+00
   3.70143570e+13 -1.16547275e+14]
 [-3.40000000e+01  0.00000000e+00 -1.67000000e+02 -3.70143570e+13
   0.00000000e+00 -9.57076369e+13]
 [ 9.60000000e+01  1.67000000e+02  0.00000000e+00  1.16547275e+14
   9.57076369e+13  0.00000000e+00]]
[-1.83824197e+16 -5.80862984e+16  7.39406422e+16 -1.90168112e+16
 -6.02265957e+16  7.56026383e+16]


array([ 2.91669803e+14,  1.03597827e+14,  2.51542428e+14, -1.10000000e+01,
        3.30000000e+02,  9.10000000e+01])

In [78]:
part2 = int(np.round(np.sum(np.linalg.solve(a, b)[0:3])))
part2

646810057104753

In [60]:
# example because something's wrong:
M2 = np.array([
    [1, 1, 0, 0, -1, 0],
    [2, 2, 2, 0, -1, -1],
    [3, 0, 3, 0, 0, -1]
])

M2[2,:3] + 3 * M2[2,3:]

array([3, 0, 0])

In [61]:
y = np.array([0, 0, 0])
w = np.array([1, 0, 0])

y + 1 * w

array([1, 0, 0])

In [62]:
# (p0 - p1) x (v0 - v1) = 0
# p0 x v0 - p0 x v1 - p1 x v0 + p1 x v1 = 0 | same for i = 2
# p0 x v0 - p0 x v2 - p2 x v0 + p2 x v2 = 0 | subtract
# p0 x v1 - p0 x v2 + p1 x v0 - p2 x v0 - p1 x v1 + p2 x v2 = 0
# p0 x v1 - p0 x v2 + p1 x v0 - p2 x v0 = p1 x v1 - p2 x v2
# p0 x (v1 - v2) - v0 x (p1 - p2) = p1 x v1 - p2 x v2

M2 = np.array([
    [1, 1, 0, 0, -1, 0],
    [2, 2, 2, 0, -1, -1],
    [3, 0, 3, 0, 0, -1]
])

p1 = M2[0][:3]
p2 = M2[1][:3]
v1 = M2[0][3:]
v2 = M2[1][3:]
p0 = y
v0 = w

dv = v1 - v2
dp = p1 - p2

np.cross(p0, dv) - np.cross(v0, dp) - (np.cross(p1, v1) - np.cross(p2, v2))

array([0, 0, 0])

In [63]:
a = np.zeros([3, 6])
b = np.zeros(3)

a[0][0] = 0
a[0][1] = -dv[2]
a[0][2] = dv[1]
a[1][0] = dv[2]
a[1][1] = 0
a[1][2] = -dv[0]
a[2][0] = -dv[1]
a[2][1] = dv[0]
a[2][2] = 0

a[0][3 + 0] = 0
a[0][3 + 1] = dp[2]
a[0][3 + 2] = -dp[1]
a[1][3 + 0] = -dp[2]
a[1][3 + 1] = 0
a[1][3 + 2] = dp[0]
a[2][3 + 0] = dp[1]
a[2][3 + 1] = -dp[0]
a[2][3 + 2] = 0

b = np.cross(p1, v1) - np.cross(p2, v2)

a, b

(array([[ 0., -1.,  0.,  0., -2.,  1.],
        [ 1.,  0.,  0.,  2.,  0., -1.],
        [ 0.,  0.,  0., -1.,  1.,  0.]]),
 array([ 0, -2,  1]))

In [64]:
# I had a wrong minus sign when calculating b